In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

For this lecture you will want to have installed additional Python packages:


In [42]:
!pip install line-profiler numexpr cython

  Using cached line_profiler-1.0.tar.gz
    100% |████████████████████████████████| 90kB 623kB/s 
  Running setup.py bdist_wheel for line-profiler
  Stored in directory: /Users/fonnescj/Library/Caches/pip/wheels/42/e9/3a/aa5d3b6dbae0c78dd50300cb0f391f8153a3d907aacf655c87
  Running setup.py bdist_wheel for numexpr
  Stored in directory: /Users/fonnescj/Library/Caches/pip/wheels/91/54/14/63794ca24979314de067bb8ffc6466cd5eee0d1c4cb36d2c4c
Successfully built line-profiler numexpr



# High Performance Python

### Speeding up statistical computations in Python using `numexpr` and `cython`

In the age of "big data" and sophisitcated Bayesian and statistical learning algorithms, many are interested in optimizing the performance of the high-level languages that we use to analyse data.

[NumPy](http://numpy.scipy.org/) gets us part of the way there on Python:

* Storage of multidimensional data
* Efficient data access
* Efficient in-memory storage
* Fast methods and functions for data manipulation

This gets us part of the way there, and for many applications it is sufficient. However, there is plenty of scope for improving Python's performance in situations where speed matters.

Pure Python and Python with NumPy are not particularly fast. Below are some recent performance benchmarks comparing several computing languages (taken directly from the [Julia website](http://julialang.org)):


<div class="figure">
<table class="benchmarks">
<colgroup>
<col class="name"></col>
<col class="relative" span="9"></col>
</colgroup>
<thead>
<tr style="background-color:#EEEEEE;"><td></td><th class="system">Fortran</th><th class="system">Julia</th><th class="system">Python</th><th class="system">R</th><th class="system">Matlab</th><th class="system">Octave</th><th class="system">Mathematica</th><th class="system">JavaScript</th><th class="system">Go</th></tr>
<tr style="background-color:#EEEEEE;"><td></td><td class="version">gcc 4.8.1
</td><td class="version">0.2</td><td class="version">2.7.3
</td><td class="version">3.0.2
</td><td class="version">R2012a
</td><td class="version">3.6.4
</td><td class="version">8.0
</td><td class="version">V8 3.7.12.22
</td><td class="version">go1
</td></tr>
</thead>
<tbody>
<tr><th>fib</th><td class="data">0.26</td><td class="data">0.91</td><td class="data">30.37</td><td class="data">411.36</td><td class="data">1992.00</td><td class="data">3211.81</td><td class="data">64.46</td><td class="data">2.18</td><td class="data">1.03</td></tr>
<tr><th>parse_int</th><td class="data">5.03</td><td class="data">1.60</td><td class="data">13.95</td><td class="data">59.40</td><td class="data">1463.16</td><td class="data">7109.85</td><td class="data">29.54</td><td class="data">2.43</td><td class="data">4.79</td></tr>
<tr><th>quicksort</th><td class="data">1.11</td><td class="data">1.14</td><td class="data">31.98</td><td class="data">524.29</td><td class="data">101.84</td><td class="data">1132.04</td><td class="data">35.74</td><td class="data">3.51</td><td class="data">1.25</td></tr>
<tr><th>mandel</th><td class="data">0.86</td><td class="data">0.85</td><td class="data">14.19</td><td class="data">106.97</td><td class="data">64.58</td><td class="data">316.95</td><td class="data">6.07</td><td class="data">3.49</td><td class="data">2.36</td></tr>
<tr><th>pi_sum</th><td class="data">0.80</td><td class="data">1.00</td><td class="data">16.33</td><td class="data">15.42</td><td class="data">1.29</td><td class="data">237.41</td><td class="data">1.32</td><td class="data">0.84</td><td class="data">1.41</td></tr>
<tr><th>rand_mat_stat</th><td class="data">0.64</td><td class="data">1.66</td><td class="data">13.52</td><td class="data">10.84</td><td class="data">6.61</td><td class="data">14.98</td><td class="data">4.52</td><td class="data">3.28</td><td class="data">8.12</td></tr>
<tr><th>rand_mat_mul</th><td class="data">0.96</td><td class="data">1.01</td><td class="data">3.41</td><td class="data">3.98</td><td class="data">1.10</td><td class="data">3.41</td><td class="data">1.16</td><td class="data">14.60</td><td class="data">8.51</td></tr>
</tbody>
</table>

<p class="caption"><b>Figure:</b>
benchmark times relative to C (smaller is better, C performance = 1.0).
</p>
</div>

So, while fast relative to some scientific compution choices (*e.g.* R, Matlab), Python sometimes needs to be tweaked in order to make it a competitive choice for implementing modern statistical methods. We will cover two approachable ways of improving the performance of Python.


## Profiling

Before you barrel ahead and prematurely optimize your Python code, it is important to understand **why** and **where** your code is slow. This is achieved by systematically accounting for the resources that your code is using, such as memory, CPU time or data transfer. This process is broadly referred to as ***Profiling***, and it allows you to identify where the performance bottlenecks in your code lie.

Here, we will concentrate on optimizing performance for **CPU-bound** problems.

There are a number of tools to help you profile your code.

### `time`

For those of you on UNIX platforms, the built-in utility `time` can be used to assess how long your code takes to run.

In [2]:
!time python ../examples/abc.py

  File "../examples/abc.py", line 27
    print abc(y, N, epsilon).mean(0)
            ^
SyntaxError: invalid syntax

real	0m0.049s
user	0m0.034s
sys	0m0.010s


The output from `time` can be interpreted as:

* `real`: elapsed (wall) time
* `user`: time spent in your code
* `sys`: time spent in system (kernel) functions

The last 2 quantities account for the cycles used by your program. The remaining `real` time is often due to waiting for information either from disk or a network connection (I/O).

Python also has a `time` module (and function) that is more rudimentary; it simply returns the time, in seconds from the Epoch (1/1/1970).

In [3]:
import time
time.time()

1448901535.842893

We can use this for profiling by differencing the times before and after running some code of interest:

In [31]:
start_time = time.time()
np.product(range(1, 100000))
end_time = time.time()

end_time - start_time

0.027576208114624023

Note, however that it does not provide a breakdown of where the code spends its time.

### IPython magic: `%timeit`, `%run` and `%prun`

IPython has three built-in "magic" functions that are useful for profiling your code. 

The `%timeit` magic executes a Python statement or expressions in a loop to see how long we expect it to take for any given call. Additionally, it repeats the loop a certain number of times, and returns the best result.

As an example, consider a Python implementation of the **trapezoidal rule**, a method from numerical analysis for approximating a definite integral. Specifically, it allows us to approximate:

$$\int_a^b f(x) dx$$

using the approximation:

$$\int_a^b f(x) dx \approx (b-a) \frac{f(b) + f(a)}{2}$$

Rather than use a single interval for this estimate, we break the interval down into $n$ subintervals, to obtain a more accurate approximation.

In [32]:
def f(x):
    return 2*x*x + 3*x + 1
      
def trapez(a, b, n):
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    
    for i in range(n):
        x += h
        sumy += f(x)
    sumy += 0.5*(f(a) + f(b))
    return sumy*h

In [33]:
trapez(1, 5, 10000)

122.6930668799846

To confirm that this works, we can compare this to the symbolic solution, using Sympy:

In [34]:
from sympy import init_printing
init_printing()
import sympy as sym

In [35]:
xs = sym.symbols('xs')

fx = 2*xs*xs + 3*xs + 1

ifx = sym.integrate(fx, (xs, 1, 5))
ifx.evalf()

In [36]:
%timeit trapez(1, 5, 10000)

100 loops, best of 3: 6.52 ms per loop


`%timeit` tries to pick suitable values for the number of loops and repeats; these values can be overriden by specifying `-n` and `-r` values, respectively.

The `%run` command with a `-p` option allows you to run complete programs under the control of the Python profiler. It writes the output to the help pane, which opens at the bottom of the page.

In [37]:
# This code redirects pager output to a regular cell
from IPython.core import page
page.page = print

In [39]:
%run -p ../examples/abc.py

mu       4.045272
sigma    1.687892
dtype: float64
         1343797 function calls (1341259 primitive calls) in 2.648 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    38564    0.618    0.000    1.135    0.000 _methods.py:76(_var)
   117724    0.409    0.000    0.409    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    77128    0.168    0.000    0.181    0.000 _methods.py:43(_count_reduce_items)
    38564    0.161    0.000    0.468    0.000 _methods.py:53(_mean)
    38564    0.155    0.000    1.307    0.000 _methods.py:122(_std)
    38565    0.144    0.000    0.144    0.000 {method 'normal' of 'mtrand.RandomState' objects}
      146    0.122    0.001    0.122    0.001 {built-in method marshal.loads}
        1    0.117    0.117    2.115    2.115 abc.py:4(abc)
   117716    0.116    0.000    0.143    0.000 numeric.py:476(asanyarray)
      146    0.063    0.000    0.063    0.000 {method 'read' of '_io.FileIO' objects}
   2

The profiling information includes the following information:

* `ncalls`: number of calls to function
* `tottime`: total time spent in the given function (excluding time in calls to sub-functions)
* `percall`: time per call
* `cumtime`: cumulative time spent in this and all subfunctions 

We can see that most of the time in this example is spent inside of core NumPy functions and methods.

The `%prun` command does a similar job for single Python expressions (like function calls).

In [40]:
%prun trapez(2, 6, 100000)

         100006 function calls in 0.095 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   100002    0.051    0.000    0.051    0.000 <ipython-input-32-262718d3c827>:1(f)
        1    0.044    0.044    0.094    0.094 <ipython-input-32-262718d3c827>:4(trapez)
        1    0.000    0.000    0.095    0.095 {built-in method builtins.exec}
        1    0.000    0.000    0.094    0.094 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
 

For even more fine-grained profiling information, we can use the line profiler (`line-profiler` package) to see how long it takes each line of a function to run.

In [43]:
%load_ext line_profiler

In [44]:
run -i ../examples/bisection.py

In [45]:
%lprun -f bisection bisection(lambda x: -2*x + 5, -10, 10)

Timer unit: 1e-06 s

Total time: 0.000253 s
File: /Users/fonnescj/Github/Bios8366/examples/bisection.py
Function: bisection at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def bisection(f, a, b, tol=1e-9, max_iter=100):
     2                                           
     3                                               # Check initial values
     4         1            5      5.0      2.0      if a >= b:
     5                                                   raise ValueError('Right value must be larger than left')
     6                                           
     7         1            4      4.0      1.6      fa, fb = f(a), f(b)
     8                                           
     9         1            1      1.0      0.4      if fa*fb > 0:
    10                                                   raise ValueError('No maximum between specified values')
    11                                          

This output makes it clear that the biggest cost is in the repeated calling of the function $f$ for which the root is being found. If we could improve the speed of this function, it would be the easiest single way of improving the performance of the code.


## Fast array expression evaluation with `numexpr`

[`numexpr`](http://code.google.com/p/numexpr/) allows array expressions to be evaluated far faster that what can be achieved in Python using Numpy arrays. `numexpr` parses a string expression and optimizes and compiles the code on the fly, using a virtual machine that includes a [Just-in-time (JIT) compiler](http://en.wikipedia.org/wiki/Just-in-time_compilation). In addition, `numexpr` offers direct support for parallel multi-threaded computations, since Python's global interpreter lock is bypassed.

> Python's global interpreter lock (GIL) ensures that only one thread runs in the interpreter at once. This simplifies many of the low-level activities, such as memory management, and allows for co-operative multi-tasking. But, since the currently-running thread holds onto the interpreter, it makes multi-core parallelization difficult.

Part the reason Python can be slow for array calculations is that it creates temporary arrays to store intermediate results from array element calculations, which wastes memory and cache. `numexpr` handles such calculations in manageable chunks, which accellerates computation.

The speedup over NumPy by using `numexpr` can be as high as 20x, but is typically in the range of 2-4x.

### Example: Computing a polynomial

In [46]:
import numpy as np

x = np.linspace(-1, 1, 1e7)

0.25*x**3 + 0.75*x**2 - 1.5*x - 2

array([  0.00000000e+00,  -4.50000045e-07,  -9.00000090e-07, ...,
        -2.50000030e+00,  -2.50000015e+00,  -2.50000000e+00])

In [47]:
%timeit 0.25*x**3 + 0.75*x**2 - 1.5*x - 2

1 loops, best of 3: 775 ms per loop


In [48]:
import numexpr as ne

ne.set_num_threads(1)

In [49]:
ne.evaluate('0.25*x**3 + 0.75*x**2 - 1.5*x - 2')

array([  0.00000000e+00,  -4.50000045e-07,  -9.00000090e-07, ...,
        -2.50000030e+00,  -2.50000015e+00,  -2.50000000e+00])

In [50]:
%timeit ne.evaluate('0.25*x**3 + 0.75*x**2 - 1.5*x - 2')

10 loops, best of 3: 116 ms per loop


`numexpr` actually expands the polynomial terms so that it does not need to use a transcendental function.

We can achieve further gains in performance by multithreading the calculations:

In [51]:
ne.set_num_threads(4)

In [52]:
%timeit ne.evaluate('0.25*x**3 + 0.75*x**2 - 1.5*x - 2')

10 loops, best of 3: 69.8 ms per loop


Since the performance of processors has outpaced that of memory in the past several decades, the CPU spends a lot of time waiting for memory to give it computations; this is the ***processor-memory performance gap***.

![performance gap](http://www.techdesignforums.com/practice/files/2013/02/tdf-snps-ARMcc-feb13-fig1lg.jpg)
(graph courtesy http://www.techdesignforums.com)

CPU caches are often used to make up for this difference. CPU caches are more effective when the data are optimally located in memory to take advantage of cache performance. `numexpr` does this by moving contiguous blocks of data from memory to the CPU cache, reusing them as much as possible within the cache to more quickly give the CPU access to the data.

### Limitations

`numexpr` only implements element-wise operations. So, `a * b` becomes:

    for i in range(N):
        c[i] = a[i] * b[i]

Similarly, it cannot index other parts of arrays in the same expression:

    for i in range(N):
        c[i] = a[i-1] + a[i] * b[i]

## Cython

Python developers typically solve performance constraints by building Python extensions by wrapping code written in other languages (for example, SciPy contains more lines of C/C++/Fortran than Python). However, programming with the Python/C API is not straightforward for most users.

Cython is a language that allows Python programmers to write fast code without having to write C/C++/Fortran directly. It looks much like Python code, but with type declarations. Cython code is translated it to C (or C++ or others), which is then compiled to create a Python extension that we can import and use. 

Using Cython, we can achieve speedups of several orders of magnitude, often *faster than hand-coded C code*. In addtion, Cython is compatible with core scientific programming tools like NumPy and IPython.

Cython has built-in support for multicore processing.

Cython is used to varying degrees by other packages in the Python scientific stack, such as pandas, sympy, scikits-learn and SciPy.

### Example: Numerical integration

Recall from above the function `trapez` for performing numerical integration using the trapezoidal rule. As a benchmark, let's time the execution of the pure-Python version of the trapezoidal rule:

In [53]:
%timeit trapez(1, 5, 1000)

1000 loops, best of 3: 644 µs per loop


Perhaps the easiest way to use Cython, is via the IPython `cythonmagic`, which allows us to run Cython interactively:

In [54]:
%load_ext cython

In [55]:
%%cython

def f(x):
    return 2*x*x + 3*x + 1

def trapez2(a, b, n):
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    for i in range(n):
        x += h
        sumy += f(x)
    sumy += 0.5*(f(a) + f(b))
    return sumy*h

The Cython magic is doing a lot of work for you: it compiles the code into an extension module, and loads it into the notebook. This allows us to ignore all of the compilation details of building Cython extensions. 

If we run `trapez2`, we can see a reasonable speedup simply by compiling it, unchanged, using Cython.

In [56]:
%timeit trapez2(1, 5, 1000)

1000 loops, best of 3: 320 µs per loop


Under the hood, several things are happening in order to deliver this improved performance. The Cython source code is translated into C source code by `cython`. Then, this C source is compiled, using the appropriate compiler, flags and associated library files (if any), into a Python extension. This extension is then loaded by IPython into the current session.

![cython flow](images/cython.png)

C extensions can also be compiled manually, using a setup file. Here is an example for an extension called `dist` within a package called `probability`:

    from distutils.core import setup
    from distutils.extension import Extension
    from Cython.Distutils import build_ext

    import numpy as np

    setup(
        cmdclass = {'build_ext': build_ext},
        ext_modules = [Extension("dist", ["probability/src/dist.pyx"], include_dirs=[np.get_include()])]
    )
    
It mainly uses machinery from a core Python package `distutils` that manages the build process.

If we look at the `trapez2` function compared to the pure Python `trapez`, we see that the Cython version appears to be a lower-level function:

In [57]:
trapez?

Signature: trapez(a, b, n)
Docstring: <no docstring>
File:      ~/Github/Bios8366/notebooks/<ipython-input-32-262718d3c827>
Type:      function


In [58]:
trapez2?

Docstring: <no docstring>
Type:      builtin_function_or_method


To get a closer look at where Cython is improving our unchanged Python code, we can add an `--annotate` flag to the `%%cython` magic declaration:

In [59]:
%%cython --annotate

def f(x):
    return 2*x*x + 3*x + 1
      
def trapez2(a, b, n):
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    for i in range(n):
        x += h
        sumy += f(x)
    sumy += 0.5*(f(a) + f(b))
    return sumy*h

In the above, the line color indicates the "typedness" of the extension, where yellower lines are closer to Python, and therefore require calls to the Python C API, while whiter lines indicate code that is closer to pure C, hence requiring few, if any, Python API calls.

If you click on a line, it unravels to show you the C code that results from the call to `cython`.

The goal in speeding up code with Cython is to turn as many lines to white as we can. The easiest way to do this is to add type declarations to the Python code:

In [60]:
%%cython --annotate

# Add type to argument
def ff(double x):
    return 2*x*x + 3*x + 1

# Add types to arguments
def trapez3(double a, double b, int n):
    # Declare types of variables
    cdef double h, x, sumy
    cdef int i
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    for i in range(n):
        x += h
        sumy += ff(x)
    sumy += 0.5*(ff(a) + ff(b))
    return sumy*h

In [61]:
%timeit trapez3(1, 5, 1000)

10000 loops, best of 3: 79.3 µs per loop


This gives us a considerable speedup. The next thing we might try is to *inline* the polynomial function. By inlining, we mean that we ask the compiler to perform an inline expansion of said function; that is, it will insert a copy of the function itself wherever the function is called, instead of calling the function wherever it is defined.

We do three things to the specification of `ff`:

* change `def` to `cdef`
* add a return type to the function
* add an `inline` keyword

In [62]:
%%cython --annotate

cdef inline double ff(double x):
    return 2*x*x + 3*x + 1

cpdef trapez4(double a, double b, int n):
    cdef double h, x, sumy
    cdef int i
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    for i in range(n):
        x += h
        sumy += ff(x)
    sumy += 0.5*(ff(a) + ff(b))
    return sumy*h

The `cdef` keyword declares a C object. Everything that follows it is therefore specified in terms of C; we are essentially writing C, but using a subset of Python's syntax rules. So, when we are creating a function `cdef ff` it is a C function, and is not available to you in Python.

`cpdef` is a hybrid declaration that creates both a C interface and a Python interface to the function.

Let's see how this performs.

In [63]:
%timeit trapez4(1, 5, 1000)

100000 loops, best of 3: 3.33 µs per loop


Woof! That's a big speedup, and there's not much yellow left in the annotated code. 

If you would like a very simple way of injecting types into your code with Cython, without modifying any of the code itelf, you can use the `@cython.locals` decorator. Note that you don't get as fast of a speedup as we have just achieved.

In [64]:
%%cython
import cython

@cython.locals(x=cython.double)
def f(x):
    return 2*x*x + 3*x + 1
     
@cython.locals(a=cython.double, b=cython.double, n=cython.int,
               h=cython.double, sumy=cython.double, i=cython.int,
               x=cython.double, func=cython.double)
def trapez5(a, b, n):
    h = (b-a)/float(n) 
    sumy = 0
    x=a
    
    for i in range(n):
        x += h
        sumy += f(x)
    sumy += 0.5*(f(a) + f(b))
    return sumy*h

In [65]:
%timeit trapez5(1, 5, 1000)

10000 loops, best of 3: 88.3 µs per loop


If you can stand to look at it, you can peek at all the C code that is generated by Cython just to optimize this short function.

In [ ]:
%load ../examples/trapezoid.c

Due to conveneince, running Cython from IPython is a preferred way of using the language. However, if we have some legacy C/C++ code that we wish to use in Python, we can do that by writing a wrapper and calling `cython` from the terminal.

Here is the C code:

In [ ]:
%load ../examples/fact.h

And here is the Cython wrapper. Cython code is stored in files with a `.pyx` extension.

In [ ]:
%load ../examples/fact.c

In [ ]:
%load ../examples/fact.pyx

In [69]:
! cython ../examples/fact.pyx

Now we can compile the extension.

In [76]:
import os
os.chdir('../examples')

!gcc -Wall -fno-strict-aliasing -static -undefined dynamic_lookup \
    -bundle -arch x86_64 \
    -I/Users/fonnescj/anaconda3/include/python3.5m \
    -o fact.so fact.c
    
os.chdir('../notebooks')

fact.c:1506:28: warning: unused function '__Pyx_PyObject_AsString' [-Wunused-function]
static CYTHON_INLINE char* __Pyx_PyObject_AsString(PyObject* o) {
                           ^
fact.c:1503:32: warning: unused function '__Pyx_PyUnicode_FromString' [-Wunused-function]
static CYTHON_INLINE PyObject* __Pyx_PyUnicode_FromString(const char* c_str) {
                               ^
fact.c:1568:26: warning: unused function '__Pyx_PyObject_IsTrue' [-Wunused-function]
static CYTHON_INLINE int __Pyx_PyObject_IsTrue(PyObject* x) {
                         ^
fact.c:1618:33: warning: unused function '__Pyx_PyIndex_AsSsize_t' [-Wunused-function]
static CYTHON_INLINE Py_ssize_t __Pyx_PyIndex_AsSsize_t(PyObject* b) {
                                ^
fact.c:1680:33: warning: unused function '__Pyx_PyInt_FromSize_t' [-Wunused-function]
static CYTHON_INLINE PyObject * __Pyx_PyInt_FromSize_t(size_t ival) {
                                ^
fact.c:1248:32: warning: unused function '__Pyx_PyInt_From_l

In [77]:
!cp ../examples/fact.so .

In [78]:
import fact

fact.fact(5)

### Using lists and arrays in Cython

The above example used only scalar variables. When we have vector-valued data, we need to declare the appropriate types. Here's a simple example, using a function that calculates the Euclidean distance between two arrays:

In [79]:
def euclidean(x, y):
    x = np.array(x)
    y = np.array(y)
    return np.sqrt(((x - y) ** 2).sum())

In [80]:
%timeit euclidean(np.random.randn(10), np.random.randn(10))

The slowest run took 18.02 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 12.8 µs per loop


In order to get a speedup under Cython, we need to iterate over the elements of each passed array, and aggregate them manually.

In [81]:
%%cython --annotate

import cython
cimport numpy as np
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def euclidean2(np.ndarray[np.float64_t, ndim=1] x, 
               np.ndarray[np.float64_t, ndim=1] y):
    cdef: 
        double diff
        int i
    diff = 0
    for i in range(x.shape[0]):
        diff += (x[i] - y[i])**2
    return sqrt(diff)


In [82]:
%timeit euclidean2(np.random.randn(10), np.random.randn(10))

The slowest run took 14.30 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 6.64 µs per loop


The decorators for `trapez5` are **compiler directives** that alter the behavior of Cython code. Setting `boundscheck` to False removes boundary checking for indexing operations, forcing us to ensure that we do not try to index arrays using index vlaues that are out of bounds. When we set `wraparound` to False, Cython will not support negative indexes, as is the case with Python. While these directives may increase the speed of our code, it can be dangerous; if we do not ensure that we index our arrays properly, it may cause segmentation faults or data corruption.

The full set of compiler directives are described in the [Cython docs](http://docs.cython.org/src/reference/compilation.html#compiler-directives).

Here is the same code using lists instead of NumPy arrays:

In [83]:
%%cython --annotate

from libc.math cimport sqrt

def euclidean3(list x, list y):
    cdef: 
        double diff
        int i
    diff = 0
    for i in range(len(x)):
        diff += (x[i] - y[i])**2
    return sqrt(diff)


In [84]:
%timeit euclidean3(np.random.randn(10).tolist(), np.random.randn(10).tolist())

The slowest run took 14.53 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 7.07 µs per loop


### `pyximport`

If we have some Cython source code, we can use `pyximport` to directly import it as if it were a Python module.

In [85]:
import pyximport

# Move source into current directory
!cp ../examples/trapezoid.pyx .

# Allow it to use Python's import mechanism
pyximport.install()

from trapezoid import trapez as trapez_pyx
trapez_pyx(1, 10, 10) 

In other words, it treats `.pyx` files as if they were `.py` files. This includes detecting changes in the source file,, if any, and recompiling it as necessary before importing.

### Benchmark example: Gibbs sampling

Let's see if we can use Cython to speed up MCMC.

Gibbs sampler for function:

$$f(x,y) = x x^2 \exp(-xy^2 - y^2 + 2y - 4x)$$

using conditional distributions:
    
$$x|y \sim Gamma(3, y^2 +4)$$
$$y|x \sim Normal(\frac{1}{1+x}, \frac{1}{2(1+x)})$$

Here is the pure Python implementation:

In [86]:
from numpy import zeros, random, sqrt
gamma = random.gamma
normal = random.normal

def pygibbs(N=20000, thin=200):
    mat = zeros((N,2))
    x,y = mat[0]
    for i in range(N):
        for j in range(thin):
            x = gamma(3, y**2 + 4)
            y = normal(1./(x+1), 1./sqrt(2*(x+1)))
        mat[i] = x,y

    return mat

In [87]:
%timeit pygibbs(1000, 10)

10 loops, best of 3: 27.1 ms per loop


Unchanged, compiling this code with Cython results in a slight improvement in speed.

In [88]:
%%cython

from numpy import zeros, random, sqrt
gamma = random.gamma
normal = random.normal

def cygibbs(N=20000, thin=200):
    mat = zeros((N,2))
    x,y = mat[0]
    for i in range(N):
        for j in range(thin):
            x = gamma(3, y**2 + 4)
            y = normal(1./(x+1), 1./sqrt(2*(x+1)))
        mat[i] = x,y

    return mat

In [89]:
%timeit cygibbs(1000, 10)

10 loops, best of 3: 21.4 ms per loop


Now, for some type declarations:

In [90]:
%%cython

from numpy import zeros, random, sqrt
from numpy cimport *
gamma = random.gamma
normal = random.normal

def cygibbs2(int N=20000, int thin=200):
    cdef: 
        ndarray[float64_t, ndim=2] mat = zeros((N,2))
        float64_t x,y = 0
        int i,j
    for i in range(N):
        for j in range(thin):
            x = gamma(3, y**2 + 4)
            y = normal(1./(x+1), 1./sqrt(2*(x+1)))
        mat[i] = x,y

    return mat

In [91]:
%timeit cygibbs2(1000, 10)

100 loops, best of 3: 20 ms per loop


A full-blown "Cythonization" involves using GSL's random number generators, and giving Cython a few more instructions:

In [92]:
# If you are using Homebrew on OS X, you can install GSL using "brew"
!brew install gsl

==> Downloading https://homebrew.bintray.com/bottles/gsl-1.16.el_capitan.bottle.2.tar.gz
########################################################################################### 100.0%
==> Pouring gsl-1.16.el_capitan.bottle.2.tar.gz
🍺  /usr/local/Cellar/gsl/1.16: 245 files, 8.2M


In [93]:
%%cython -lm -lgsl -lgslcblas

cimport cython
import numpy as np
from numpy cimport *

cdef extern from "math.h":
    double sqrt(double) 
  
cdef extern from "gsl/gsl_rng.h":
    ctypedef struct gsl_rng_type
    ctypedef struct gsl_rng

    gsl_rng_type *gsl_rng_mt19937
    gsl_rng *gsl_rng_alloc(gsl_rng_type * T) nogil
  
cdef extern from "gsl/gsl_randist.h":
    double gamma "gsl_ran_gamma"(gsl_rng * r,double,double)
    double gaussian "gsl_ran_gaussian"(gsl_rng * r,double)
  
cdef gsl_rng *r = gsl_rng_alloc(gsl_rng_mt19937)

@cython.wraparound(False)
@cython.boundscheck(False)
def gibbs(int N=20000,int thin=500):
    cdef: 
        double x=0
        double y=0
        int i, j
        ndarray[float64_t, ndim=2] samples

    samples = np.empty((N,thin))
    for i from 0 <= i < N:
        for j from 0 <= j < thin:
            x = gamma(r,3,1.0/(y*y+4))
            y = gaussian(r,1.0/sqrt(x+1))
        samples[i,0] = x
        samples[i,1] = y
    return samples

In [94]:
%timeit gibbs(1000, 10)

1000 loops, best of 3: 1.26 ms per loop


## Exercise

Try using Cython to improve the performance of the gradient descent algorithm from our optimization lecture:

In [95]:
from scipy import optimize

def gradient_descent(x0, f, f_prime, adapt=False):
    x_i, y_i = x0
    all_x_i = list()
    all_y_i = list()
    all_f_i = list()

    for i in range(1, 100):
        all_x_i.append(x_i)
        all_y_i.append(y_i)
        all_f_i.append(f([x_i, y_i]))
        dx_i, dy_i = f_prime(np.asarray([x_i, y_i]))
        if adapt:
            # Compute a step size using a line_search
            step = optimize.line_search(f, f_prime,
                                np.r_[x_i, y_i], -np.r_[dx_i, dy_i],
                                np.r_[dx_i, dy_i], c2=.05)
            step = step[0]
        else:
            step = 1
        x_i += -step*dx_i
        y_i += -step*dy_i
        if np.abs(all_f_i[-1]) < 1e-16:
            break
    return all_x_i, all_y_i, all_f_i

In [100]:
x0, y0 = 1.6, 1.1

f, f_prime = quad(0.8)
%timeit gd_x_i, gd_y_i, gd_f_i = gradient_descent([x0, y0], f, f_prime)

The slowest run took 4.37 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 1.55 ms per loop


Here is a sample function to optimize. Recall from Section 3 that it returns both the quadratic function and its gradient.

In [96]:
def quad(epsilon, ndim=2):
    def f(x):
       x = np.asarray(x)
       y = x.copy()
       y *= np.power(epsilon, np.arange(ndim))
       return .33*np.sum(y**2)

    def f_prime(x):
       x = np.asarray(x)
       y = x.copy()
       scaling = np.power(epsilon, np.arange(ndim))
       y *= scaling
       return .33*2*scaling*y

    return f, f_prime

In [ ]:
# Write answer here

## References

[A guide to analyzing Python performance](http://www.huyng.com/posts/python-performance-analysis/)

[Kurt Smith's Cython tutorial from SciPy 2013](https://www.youtube.com/watch?v=JKCjsRDffXo)
